Laen sisse test data

Teen selle testimise kasutades juhendeid siin: https://docs.databricks.com/spark/latest/graph-analysis/graph-analysis-graphx-tutorial.html

In [1]:
val twitterDF = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/home/jovyan/data/kagandi_twitter/twitter.csv")

twitterDF = [src: int, dst: int]


[src: int, dst: int]

In [2]:
twitterDF.count()

16011444

In [3]:
val twitter_fakes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/home/jovyan/data/kagandi_twitter/twitter_fake_ids.csv")

twitter_fakes = [id: int,  label: string]


[id: int,  label: string]

In [4]:
twitter_fakes.count()

12437

## Edasi

  * Teha 

Importing graph stuff from graphX

In [5]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

In [6]:
val users = twitterDF.rdd.distinct()
    .flatMap(x => Iterable(x(0).asInstanceOf[Number].longValue, x(1).asInstanceOf[Number].longValue))
    .distinct()
    .toDF()

users = [value: bigint]


[value: bigint]

In [7]:
val userVertices: RDD[(VertexId, String)] = users
   .rdd
   .map(row => (row(0).asInstanceOf[Number].longValue, "Real"))


userVertices = MapPartitionsRDD[45] at map at <console>:37


MapPartitionsRDD[45] at map at <console>:37

In [8]:
val userEdges:RDD[Edge[Long]] = twitterDF.rdd
   .map(row => Edge(row(0).asInstanceOf[Number].longValue, row(1).asInstanceOf[Number].longValue, 1))

userEdges = MapPartitionsRDD[46] at map at <console>:34


MapPartitionsRDD[46] at map at <console>:34

In [9]:
val defaultUser = ("Jon Doe")

defaultUser = Jon Doe


Jon Doe

In [10]:
val usersGraph = Graph(userVertices, userEdges, defaultUser)

usersGraph = org.apache.spark.graphx.impl.GraphImpl@618d1714


org.apache.spark.graphx.impl.GraphImpl@618d1714

In [11]:
usersGraph.cache()

org.apache.spark.graphx.impl.GraphImpl@618d1714

In [12]:
println("Total Number of Users: " + usersGraph.numVertices)
println("Total Number of Relationships: " + usersGraph.numEdges)
// sanity check
println("Total Number of Relationships in Original Data: " + twitterDF.count)

Total Number of Users: 5384162
Total Number of Relationships: 16011444
Total Number of Relationships in Original Data: 16011444


# PageRank-i jooksutamine

In [13]:
val ranks = usersGraph.pageRank(0.0001).vertices

ranks = VertexRDDImpl[942] at RDD at VertexRDD.scala:57


VertexRDDImpl[942] at RDD at VertexRDD.scala:57

In [14]:
ranks.sortBy(_._2, ascending=false).take(10)

Array((10307,117.27534996155201), (27459,80.5349580145939), (15226,72.08546700606429), (16006,64.9225696711577), (8713,64.25478467976535), (426,63.792155159804025), (4241,63.24161136755771), (8694,59.45448340145653), (120203,55.93105619475406), (10225,55.78781743270077))

## Teha siin ümber et näitaks kasutaja infot ja nende klassi (real/fake)

In [44]:
ranks
  .join(stationVertices)
  .sortBy(_._2._1, ascending=false) // sort by the rank
  .take(10) // get the top 10
  .foreach(x => println("Station: " + x._2._2 + "\t\t Rank: " + x._2._1))

Station: San Jose Diridon Caltrain Station		 Rank: 3.95332475313009
Station: San Francisco Caltrain (Townsend at 4th)		 Rank: 3.2708212384471804
Station: Mountain View Caltrain Station		 Rank: 2.5131577090624435
Station: Redwood City Caltrain Station		 Rank: 2.2289235741998996
Station: San Francisco Caltrain 2 (330 Townsend)		 Rank: 2.17767954644027
Station: Harry Bridges Plaza (Ferry Building)		 Rank: 1.781452831653251
Station: 2nd at Townsend		 Rank: 1.544284802694958
Station: Santa Clara at Almaden		 Rank: 1.5342586021027171
Station: Townsend at 7th		 Rank: 1.5309106362489386
Station: Embarcadero at Sansome		 Rank: 1.5045403211302477


Most common trips between station to station

In [28]:
stationGraph
  .groupEdges((edge1, edge2) => edge1 + edge2)
  .triplets
  .sortBy(_.attr, ascending=false)
  .map(triplet =>
    "There were " + triplet.attr.toString + " trips from " + triplet.srcAttr + " to " + triplet.dstAttr + ".")
  .take(10)
  .foreach(println)

There were 614 trips from San Francisco Caltrain 2 (330 Townsend) to Townsend at 7th.
There were 584 trips from Townsend at 7th to San Francisco Caltrain 2 (330 Townsend).
There were 573 trips from San Francisco Caltrain 2 (330 Townsend) to Townsend at 7th.
There were 555 trips from Townsend at 7th to San Francisco Caltrain 2 (330 Townsend).
There were 555 trips from San Francisco Caltrain 2 (330 Townsend) to Townsend at 7th.
There were 550 trips from Townsend at 7th to San Francisco Caltrain (Townsend at 4th).
There were 525 trips from Townsend at 7th to San Francisco Caltrain 2 (330 Townsend).
There were 513 trips from Townsend at 7th to San Francisco Caltrain (Townsend at 4th).
There were 511 trips from Harry Bridges Plaza (Ferry Building) to Embarcadero at Sansome.
There were 506 trips from San Francisco Caltrain 2 (330 Townsend) to Townsend at 7th.


In and out degrees

In [30]:
stationGraph
    .inDegrees
    .join(stationVertices)
    .sortBy(_._2._1, ascending=false)
    .take(10)
    .foreach(x => println(x._2._2 + " has " + x._2._1 + " in degrees."))

San Francisco Caltrain (Townsend at 4th) has 34166 in degrees.
San Francisco Caltrain 2 (330 Townsend) has 22109 in degrees.
Harry Bridges Plaza (Ferry Building) has 17403 in degrees.
2nd at Townsend has 15279 in degrees.
Townsend at 7th has 15126 in degrees.
Embarcadero at Sansome has 14705 in degrees.
Market at Sansome has 13778 in degrees.
Steuart at Market has 13383 in degrees.
Temporary Transbay Terminal (Howard at Beale) has 12748 in degrees.
Market at 10th has 9988 in degrees.


In [31]:
stationGraph
  .outDegrees // out degrees
  .join(stationVertices)
  .sortBy(_._2._1, ascending=false)
  .take(10)
  .foreach(x => println(x._2._2 + " has " + x._2._1 + " out degrees."))

San Francisco Caltrain (Townsend at 4th) has 25631 out degrees.
San Francisco Caltrain 2 (330 Townsend) has 21245 out degrees.
Harry Bridges Plaza (Ferry Building) has 16917 out degrees.
Temporary Transbay Terminal (Howard at Beale) has 14242 out degrees.
Embarcadero at Sansome has 13885 out degrees.
2nd at Townsend has 13746 out degrees.
Steuart at Market has 13478 out degrees.
Townsend at 7th has 13465 out degrees.
Market at 10th has 11449 out degrees.
Market at Sansome has 11317 out degrees.


Highest in/out degree ratio stations

In [32]:
stationGraph
  .inDegrees
  .join(stationGraph.outDegrees) // join with out Degrees
  .join(stationVertices) // join with your other stations
  .map(x => (x._2._1._1.toDouble/x._2._1._2.toDouble, x._2._2)) // ratio of in to out
  .sortBy(_._1, ascending=false)
  .take(5)
  .foreach(x => println(x._2 + " has a in/out degree ratio of " + x._1))

Redwood City Medical Center has a in/out degree ratio of 1.5333333333333334
San Mateo County Center has a in/out degree ratio of 1.4724409448818898
SJSU 4th at San Carlos has a in/out degree ratio of 1.3621052631578947
San Francisco Caltrain (Townsend at 4th) has a in/out degree ratio of 1.3329952011236395
Paseo de San Antonio has a in/out degree ratio of 1.2535046728971964


Lowest in/out degree ratio stations

In [33]:
stationGraph
  .inDegrees
  .join(stationGraph.inDegrees) // join with out Degrees
  .join(stationVertices) // join with your other stations
  .map(x => (x._2._1._1.toDouble/x._2._1._2.toDouble, x._2._2)) // ratio of in to out
  .sortBy(_._1)
  .take(5)
  .foreach(x => println(x._2 + " has a in/out degree ratio of " + x._1))


Beale at Market has a in/out degree ratio of 1.0
SJSU - San Salvador at 9th has a in/out degree ratio of 1.0
Santa Clara County Civic Center has a in/out degree ratio of 1.0
Embarcadero at Vallejo has a in/out degree ratio of 1.0
Castro Street and El Camino Real has a in/out degree ratio of 1.0
